In [1]:
import os
import time
import pandas as pd
import numpy as np

import geopandas as gpd
import rioxarray as rioxr
import rasterio

#from skimage.filters.rank import entropy
import sample_rasters as sr
from rasterio.crs import CRS

from shapely.geometry import box

import planetary_computer as pc

In [2]:
# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

itemids = pd.read_csv(os.path.join(os.getcwd(),'temp', 'aoi_naip_itemids.csv'))

csv_name = 'spectral_window_test_set.csv'
all_pts = pd.read_csv(os.path.join(os.getcwd(), csv_name))

# ***************************************************
# ***************************************************

In [3]:
# temporary folder for aux rasters
folp = os.path.join(os.getcwd(),'temp','aux_naip_rasters')
if os.path.exists(folp) == False:
    os.mkdir(folp)

In [21]:
t0 = time.time()

sampled_pts = []
for i in [9]:
#for i in range(len(itemids)):
    # ---------------------------------------
    # open raster reader for NAIP scene
    itemid = itemids.itemid[i]
    item = sr.get_item_from_id(itemid)    
    href = pc.sign(item.assets["image"].href)
    naip_rast_r = rioxr.open_rasterio(href) 

    # ---------------------------------------
    # find polygons for that NAIP scene
    poly_fp = sr.path_to_polygons(itemids.iloc[i].aoi_name, itemids.iloc[i].year)
    polys = gpd.read_file(poly_fp)
    
    polys_pts = []
    # iterate through polygons in scene
    for j in [0]:
#    for j in range(len(polys)):
        # ---------------------------------------
        # create enlarged bounding box
        poly = polys.geometry[j]
        bbox = box(*poly.bounds)  
        # NEED TO MAKE bbox BIGGER
        reduce = gpd.GeoDataFrame({'geometry':[bbox]}, crs=polys.crs)
        reduce = reduce.to_crs(naip_rast_r.rio.crs)      

        # ---------------------------------------
        # clip NAIP scene
        rast = naip_rast_r.rio.clip_box(*reduce.total_bounds)
        
        # ---------------------------------------
        # save auxiliary entropy rasters for R,G,B,NIR bands of clipped scene
        band_names = ['r_', 'g_', 'b_', 'nir_']
        tags = ['_entrs']
        window_fps = []
        window_cols = []
        
        for band_name, band_n in zip(band_names,range(1,5)):
            rast_name = band_name + itemid + '_poly_'+str(j)
            sr.entropy_raster(raster = rast, band=band_n, rast_name=rast_name, n=3, folder_path=folp)

            for tag in tags:
                window_fps.append(os.path.join(folp, rast_name + tag + '.tif'))        
                window_cols.append( band_name.replace('_','')+tag.replace('s',''))
                
        # ------------------------------
        # make auxiliary NDVI of clipped scene
        t0 = time.time()
        red_band = rast.sel(band=1).astype('int16') 
        nir_band = rast.sel(band=4).astype('int16')
        ndvi = ((nir_band - red_band) / (nir_band + red_band)*100)+100
        
        # make auxiliary NDVI entropy
        band_names.append('ndvi_')
        rast_name = 'ndvi_'+itemid
        sr.entropy_raster(rast_data=ndvi.astype('uint8'),
                          crs=rast.rio.crs, 
                          transf=rast.rio.transform(), 
                          rast_name=rast_name, 
                          n=3, 
                          folder_path=folp)

        for tag in tags:
            window_fps.append(os.path.join(folp, rast_name + tag + '.tif'))        
            window_cols.append( 'ndvi'+tag.replace('s',''))
        
        # ---------------------------------------
        # find points in current polygon
        pts_poly = all_pts.loc[ (all_pts['naip_id'] == itemid) & (all_pts['polygon_id'] == j)]
        crs = CRS.from_string(pts_poly.pts_crs.iloc[0])
        pts_poly_df = sr.geodataframe_from_csv(df = pts_poly, lon_label='x', lat_label='y', crs=crs)
        pts_col = pts_poly_df.to_crs(naip_rast_r.rio.crs).geometry

        # ---------------------------------------
        # sample raster values for points in current polygon
        samples = []
        for fp, col_name in zip(window_fps, window_cols):
            rast_r = rasterio.open(fp)
            sample = sr.sample_raster_from_pts(pts_col, rast_r, [col_name])    
            samples.append(sample)
            
        # ---------------------------------------
        # Add all derived spectral data to pts dataframe
        new_features = pd.concat(samples, axis = 1)
        pts = pd.concat([pts_poly, new_features.set_index(pts_poly_df.index)], axis=1)                
        
        # -----------------------------
        # collect all points from each polygon in the scene
        polys_pts.append(pts)
        
        # ---------------------------------------
        # delete aux entropy rasters
        for fp in window_fps:
            os.remove(fp)
            
sampled_pts= pd.concat(polys_pts).sort_index()
        
print(time.time() -t0)

0.39343714714050293


In [22]:
sampled_pts

,x,y,pts_crs,aoi,naip_id,polygon_id,r,r_max,r_min,r_diff,...,min_lidar,min_max_diff,avg_lidar,iceplant,geometry,r_entr,g_entr,b_entr,nir_entr,ndvi_entr
113247,238505.152592,3.810782e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_060_20200521,0,71,76,70,6,...,0.0,6.0,1.666667,1.0,POINT (238505.153 3810781.681),3.012816,3.190239,2.098398,4.185230,3.224047
113248,238487.179744,3.810790e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_060_20200521,0,81,86,79,7,...,0.0,4.0,1.888889,1.0,POINT (238487.180 3810790.447),3.288678,2.914376,1.971594,4.004364,3.177669
113249,238490.218603,3.810767e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_060_20200521,0,89,100,84,16,...,1.0,5.0,3.666667,1.0,POINT (238490.219 3810767.430),3.551970,2.461983,3.248635,3.689902,3.481562
113253,238505.489687,3.810774e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_060_20200521,0,70,79,70,9,...,1.0,5.0,2.777778,1.0,POINT (238505.490 3810773.852),2.969329,2.815937,2.856633,4.030395,3.633506
113282,238506.469535,3.810770e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_060_20200521,0,70,76,68,8,...,1.0,2.0,1.777778,1.0,POINT (238506.470 3810769.706),3.081782,2.019274,2.593585,3.581445,3.314709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126776,238469.247548,3.810785e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_060_20200521,0,84,85,80,5,...,2.0,3.0,3.222222,1.0,POINT (238469.248 3810784.711),3.348518,3.345074,2.442840,4.185230,3.259204
126783,238475.454220,3.810784e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_060_20200521,0,82,83,75,8,...,2.0,3.0,3.222222,1.0,POINT (238475.454 3810783.548),3.276109,3.052308,3.374548,4.116265,3.745406
126845,238498.256632,3.810770e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_060_20200521,0,73,87,73,14,...,1.0,5.0,2.777778,1.0,POINT (238498.257 3810769.668),3.676441,3.745406,3.198017,4.073330,3.509036
126866,238479.981759,3.810792e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_060_20200521,0,82,87,80,7,...,0.0,4.0,1.888889,1.0,POINT (238479.982 3810792.309),2.943851,2.801924,1.638784,3.219161,2.662550
